# Optimal thermal control with linprog

Pierre Haessig, May 2016

In [ ]:
from __future__ import unicode_literals, print_function, division

In [ ]:
import numpy as np
import cvxpy as cx
cx.__version__

notice: the cvxpy API may change in the forthcoming 1.0 version (cf. [issue #199](https://github.com/cvxgrp/cvxpy/issues/199))

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
mpl.style.use(['seaborn-deep', 'seaborn-whitegrid', 'seaborn-notebook'])
mpl.rcParams['figure.figsize'] = [8, 3.5]  # thin plots for time series

## Problem definition

Load problem data. choice of units: hours, kW, Celsius degrees.

In [ ]:
from therm_data import dt, C, R, P_max

In [ ]:
print('Timestep dt: {} h'.format(dt))
print('Capacity C: {} kWh/K'.format(C))
print('Resistance R: {} °C/kW'.format(R))
print('P_max: {} kWh'.format(P_max))

In [ ]:
N_sim = int(24/dt)

dynamics in discrete time (backward Euler), with timestep $\Delta_t$ :

$$T^+ = T + \frac{\Delta_t}{C} ( P - \frac{T-T_{out}}{R} ) $$

In [ ]:
from therm_data import T0, T_out, occupancy, T_abs, T_pres

In [ ]:
t = np.arange(N_sim)*dt
occ = occupancy(t)
T_min = np.zeros(N_sim) + T_abs # °C
T_min[occ] = T_pres

## LP problem

with hard temperature constraint $T ≥ T_{min}$

Optimization variables:

In [ ]:
P = cx.Variable(N_sim)
T = cx.Variable(N_sim)

Objective: minimize total consumption

In [ ]:
objective = cx.Minimize(cx.sum_entries(P))
objective

Build the constraints

In [ ]:
# Heating limits:
c_Pmin = P >= 0
c_Pmax = P <= P_max
constraints = [c_Pmin, c_Pmax]

# min Temperature set point
c_Tmin = T >= T_min
constraints += [c_Tmin]

# Temp dynamics
deltaT = dt/C*(P[:-1] - (T[:-1]-T_out)/R)
c_dyn = T[1:] == T[:-1] + deltaT
constraints += [c_dyn]

# Initial temperature
c_T0 = T[0] == T0
constraints += [c_T0]


print('Constraints:')
for c in constraints:
    print(c.OP_NAME, end='')
    print(' ' +  repr(c.args[0]))
    print('   ' +  repr(c.args[1]))
    print()

Solve the Linear Program:

In [ ]:
prob = cx.Problem(objective, constraints)
result = prob.solve()

print('energy consum: {:.6f} kW avg'.format(result/N_sim))

In [ ]:
def plot_traj(t, T, T_out, T_min, P):
    'plot of a simulation (temperature and heating)'
    fig, (ax1, ax2) = plt.subplots(2,1, sharex=True, figsize=(6,4))
    
    ax1.plot(t, T_min, '--', label='T_min')
    ax1.plot(t, T, label='T')
    ax1.legend(loc='upper left')

    ax2.plot(t, P, 'r')

    ax1.set(
        ylabel='Temp (°C)',
        ylim=(T_abs-0.5, T_pres+0.5)
    )
    ax2.set(
        xlabel='t (h)',
        ylabel='P (kW)',
        ylim=(P_max*-.05, P_max*1.05)
    )
    fig.tight_layout()
    
    return fig, (ax1, ax2)

In [ ]:
fig, _ = plot_traj(t, T.value, T_out, T_min, P.value);
#fig.savefig('sim_linprog.pdf')

Save result (and convert 2D column matrices into 1D arrays)

In [ ]:
np.savez('thermal_lp_sol.npz',
         T = np.array(T.value)[:, 0], 
         P = np.array(P.value)[:, 0],
         T_out = T_out,
         T_min = T_min
        )

---


## Relaxation of the min temperature constraint

The min temperature constraint
$$T ≥ T_{min} $$

is replaced by

$$ T ≥ T_{min} - T_{diff}$$
where slack variable $T_{diff} ≥0$ is heavily penalized

In [ ]:
P_rlx = cx.Variable(N_sim)
T_rlx = cx.Variable(N_sim)
# new slack variable:
T_diff = cx.Variable(N_sim)

In [ ]:
comfort_weight=10

objective_rlx = cx.Minimize(cx.sum_entries(P_rlx) +\
                        comfort_weight*cx.sum_entries(T_diff))
objective_rlx

Build the constraints

In [ ]:
# Heating limits:
constraints_rlx  = [P_rlx >= 0, P_rlx <= P_max]
# min Temperature set point, RELAXED
constraints_rlx  += [T_rlx >= T_min - T_diff]
constraints_rlx  += [T_diff >= 0]
# Temp dynamics
deltaT = dt/C*(P_rlx[:-1] - (T_rlx[:-1]-T_out)/R)
constraints_rlx  += [T_rlx[1:] == T_rlx[:-1] + deltaT]
# T0
constraints_rlx  += [T_rlx[0] == T0]

print('Constraints:')
for c in constraints_rlx :
    print(c.OP_NAME, end='')
    print(' ' +  repr(c.args[0]))
    print('   ' +  repr(c.args[1]))
    print()

Solve the Linear Program:

In [ ]:
prob_rlx  = cx.Problem(objective_rlx , constraints_rlx )
result_rlx  = prob_rlx .solve()
result_rlx 

In [ ]:
ccom = T_diff.value.mean()*comfort_weight
cener = P_rlx.value.mean()
print('energy consum: {:.3f} kW avg'.format(cener))
print('comfort violation penal: {:.3f} (with confort weight {} kW/°)'.format(
        ccom, comfort_weight))
print('LP avg cost: {:.6f}'.format(result_rlx /N_sim))

Difference in the result:

In [ ]:
result_rlx - result

In [ ]:
print('max T_min violation: {:.4f} °C'.format(T_diff.value.max()))

In [ ]:
plot_traj(t, T_rlx.value, T_out, T_min, P_rlx.value);

### Zoom at the difference

In [ ]:
t_zoom = (21.5, 22.5) # h

In [ ]:
plt.plot(t, T.value, 'kd--')
plt.plot(t, T_rlx.value, 'd-')
plt.plot(t, T_min)
plt.xlim(*t_zoom)
plt.ylim(T_pres-0.5, T_pres+0.2);

In [ ]:
plt.plot(t, T_diff.value)
plt.xlim(*t_zoom);

In [ ]:
plt.plot(t, P.value, 'd-', lw=0.8);
plt.plot(t, P_rlx.value, 'rd-')
plt.xlim(*t_zoom);

Observation:

with `comfort_weight = 10`, there is a small trade off of energy against comfort near the end of the problem (-0.063 °C loss at 21.9 h). Energy consumption is slighlty reduced, but total LP cost is the same however (at least 4 digits)

with `comfort_weight = 100`, it becomes negligible

## Closing the loop

To be done...